In [ ]:
import torch

In [ ]:
def set_device():
  if torch.cuda.is_available():
    device = torch.device("cuda")
    return device
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
  else:
    device = torch.device("cpu")
    return device
  print(device)

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
def custom_collate_fn(batch):

  global tokenizer_bert

  input_list, target_list = [a[0] for a in batch], [a[1] for a in batch]
  
  tensorized_input = tokenizer_bert(
      input_list,
      add_special_tokens=True,
      padding="longest",
      truncation = "longest_first",
      is_split_into_words=False,
      return_tensors="pt"
      )
  
  tensorized_label = torch.tensor(target_list)
  
  return tensorized_input, tensorized_label

In [ ]:
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class CustomDataset(Dataset):

  def __init__(self, input_data:list, target_data:list) -> None:
      self.X = input_data
      self.Y = target_data

  def __len__(self):
      return len(self.Y)

  def __getitem__(self, index):
      return self.X[index], self.Y[index]

In [ ]:
class CustomClassifier(nn.Module):

  def __init__(self, hidden_size: int, n_label: int):
    super(CustomClassifier, self).__init__()

    self.bert = BertModel.from_pretrained("klue/bert-base")

    dropout_rate = 0.1
    linear_layer_hidden_size = 32

    self.classifier = nn.Sequential(
        nn.Linear(hidden_size,linear_layer_hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(linear_layer_hidden_size,n_label)
    )
  
  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
    outputs = self.bert(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids,
    )
    cls_token_last_hidden_states = outputs['pooler_output']

    logits = self.classifier(cls_token_last_hidden_states)

    return logits